In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'notebook'
from sklearn.datasets import fetch_covtype
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA
from matplotlib.colors import ListedColormap
from sklearn.neighbors import KNeighborsClassifier

sns.set_theme(style='darkgrid')

# Carregar o dataset
print("Carregando o dataset 'Covertype'...\n")
data = fetch_covtype(as_frame=True)
description_data = fetch_covtype()

# Exibir descrição da base
print("Descrição do Dataset:\n")
print(description_data.DESCR)  #

# Separar variáveis e classes
X = data.data
y = data.target

# Mostrar as classes presentes no target
print("\nClasses (Tipos de Cobertura Florestal):\n")
print("Valores únicos encontrados na variável de saída (y):", sorted(y.unique()))
print(f"Total de classes distintas: {len(y.unique())}")

# Listar as features disponíveis
print("\nLista de Features Disponíveis no Dataset:\n")
for i, feature in enumerate(X.columns, 1):
    print(f"{i:2}. {feature}")

In [ ]:
features_cont = [
    'Elevation', 'Aspect', 'Slope',
    'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
    'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
    'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points'
]

print("\nVariáveis CONTÍNUAS selecionadas (10):")
print(", ".join(features_cont))

features_bin = [c for c in X.columns if c not in features_cont]
print(f"Variáveis BINÁRIAS detectadas ({len(features_bin)}): "
      f"{', '.join(features_bin[:6])} …")

scaler = StandardScaler()
X_cont      = X[features_cont]
X_cont_norm = pd.DataFrame(scaler.fit_transform(X_cont), columns=features_cont)

summary = pd.concat([
    X_cont.describe().T[['mean', 'std', 'min', 'max']].round(1).add_suffix('_orig'),
    X_cont_norm.describe().T[['mean', 'std', 'min', 'max']].round(2).add_suffix('_norm')
], axis=1)

print("\nResumo estatístico (original × normalizado):")
print(summary)

for col in features_cont:
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    fig.suptitle(f'{col} - Original vs Normalizado')

    # Histograma original
    axes[0].hist(X_cont[col], bins=30, edgecolor='black')
    axes[0].set_title('Original')
    axes[0].set_xlabel(col)
    axes[0].set_ylabel('Frequência')

    # Histograma normalizado
    axes[1].hist(X_cont_norm[col], bins=30, edgecolor='black')
    axes[1].set_title('Normalizado (StandardScaler)')
    axes[1].set_xlabel(f'{col} (normalizado)')
    axes[1].set_ylabel('Frequência')

    plt.tight_layout()
    plt.show()

print("""
Por que normalizar?
- KNN baseia‑se em DISTÂNCIA Euclidiana. Escalas muito diferentes distorcem a métrica.
- As variáveis binárias (0/1) já estão padronizadas e entram sem transformação.
""")

In [ ]:
print("Bloco de Análise Exploratória\n")

print("=== Histograma por Classe ===")
print("""
Objetivo: visualizar como cada variável contínua se distribui entre as diferentes classes (tipos de cobertura florestal).

→ Se as curvas estiverem bem separadas por classe, a variável pode ajudar o modelo.
→ Se houver muita sobreposição entre as cores, essa variável tem baixo poder discriminativo.
""")

for feature in features_cont:
    plt.figure(figsize=(8, 4))
    sns.histplot(data=X.assign(Classe=y), x=feature, hue='Classe', kde=True, bins=30, palette='tab10', element='step')
    plt.title(f'Distribuição de {feature} por Classe')
    plt.xlabel(feature)
    plt.ylabel('Frequência')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

print("=== Contour Plot com KNN (Elevation x Slope) ===")
print("""
Objetivo: visualizar o limite de decisão do KNN com base em duas variáveis contínuas (Elevation e Slope).
→ Útil para entender onde o modelo consegue separar bem as classes e onde há sobreposição.
""")

X_2d = X[['Elevation', 'Slope']]
y_2d = y

scaler_2d = StandardScaler()
X_2d_scaled = scaler_2d.fit_transform(X_2d)

sample_idx = np.random.choice(len(X_2d_scaled), size=10000, replace=False)
X_sample = X_2d_scaled[sample_idx]
y_sample = y_2d.iloc[sample_idx]

knn = KNeighborsClassifier(n_neighbors=5, weights='distance')
knn.fit(X_sample, y_sample)

x_min, x_max = X_sample[:, 0].min() - 0.5, X_sample[:, 0].max() + 0.5
y_min, y_max = X_sample[:, 1].min() - 0.5, X_sample[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300), np.linspace(y_min, y_max, 300))
Z = knn.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

plt.figure(figsize=(10, 8))
cmap = ListedColormap(sns.color_palette("tab10").as_hex())
plt.contourf(xx, yy, Z, alpha=0.3, cmap=cmap)
plt.scatter(X_sample[:, 0], X_sample[:, 1], c=y_sample, cmap=cmap, edgecolor='k', s=5)
plt.title('Contour Plot — KNN (Elevation vs. Slope)')
plt.xlabel('Elevation (normalizado)')
plt.ylabel('Slope (normalizado)')
plt.grid(True, alpha=0.4)
plt.tight_layout()
plt.show()

print("=== Análise Exploratória com Pares de Variáveis ===")
print("""
Objetivo: verificar visualmente quais pares de variáveis contínuas mostram melhor separação entre as classes.

→ Isso auxilia na escolha de variáveis mais relevantes para modelos baseados em distância.
""")

feature_pairs = [
    ('Elevation', 'Slope'),
    ('Aspect', 'Slope'),
    ('Elevation', 'Horizontal_Distance_To_Hydrology'),
    ('Hillshade_Noon', 'Hillshade_3pm'),
    ('Horizontal_Distance_To_Fire_Points', 'Vertical_Distance_To_Hydrology')
]

for f1, f2 in feature_pairs:
    plt.figure(figsize=(8, 6))
    plt.scatter(X[f1], X[f2], c=y, cmap='tab10', s=5)
    plt.colorbar(label='Classe real')
    plt.title(f'Visualização — {f1} vs {f2}')
    plt.xlabel(f1)
    plt.ylabel(f2)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

print("""
Análise dos pares de variáveis:

1. Elevation vs Slope — melhor separação entre classes. Padrões claros ao longo da elevação.
2. Aspect vs Slope — alta sobreposição. Aspect é uma variável circular (0–360º), o que dificulta a separação.
3. Elevation vs Horizontal_Distance_To_Hydrology — algum padrão, mas ainda com sobreposição significativa.
4. Hillshade_Noon vs Hillshade_3pm — forte correlação entre variáveis, separação visual fraca.
5. Horizontal_Distance_To_Fire_Points vs Vertical_Distance_To_Hydrology — agrupamentos localizados, mas com muito ruído.

→ Conclusão: Elevation + Slope foi o par com melhor desempenho visual para separação de classes.
""")

print("=== PCA 2D — Usando Todas as 54 Variáveis ===")
print("""
Objetivo: reduzir a dimensionalidade e visualizar a separação global entre classes no espaço PCA com todas as variáveis (contínuas + binárias).
""")

scaler_full = StandardScaler()
X_scaled = scaler_full.fit_transform(X)

pca_full = PCA(n_components=2)
X_pca_full = pca_full.fit_transform(X_scaled)

plt.figure(figsize=(10, 8))
plt.scatter(X_pca_full[:, 0], X_pca_full[:, 1], c=y, cmap='tab10', s=5)
plt.colorbar(label='Classe Real')
plt.title('PCA 2D — Todas as Features (54 dimensões)')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.grid(True, alpha=0.5)
plt.tight_layout()
plt.show()

print("=== PCA 2D — Somente com as 10 Variáveis Contínuas ===")
print("""
Objetivo: aplicar PCA apenas nas variáveis contínuas para eliminar o ruído causado pelas variáveis binárias, já que o PCA é uma técnica orientada por distâncias e variâncias, e variáveis binárias têm baixa variância.

O PCA busca as direções de maior variação nos dados, e as variáveis binárias, que geralmente possuem apenas dois valores (0 ou 1), não oferecem uma variação significativa para o modelo. Além disso, o PCA pode não capturar a relação entre as variáveis binárias e outras variáveis, resultando em distorções ou ruídos durante a redução dimensional. Portanto, ao aplicar o PCA nas variáveis contínuas, garantimos que a técnica se concentre nas direções mais relevantes, preservando a maior parte da variabilidade nos dados.
""")

X_continuas = X[features_cont]
scaler_cont = StandardScaler()
X_cont_scaled = scaler_cont.fit_transform(X_continuas)

pca_cont = PCA(n_components=2)
X_pca_cont = pca_cont.fit_transform(X_cont_scaled)

plt.figure(figsize=(10, 8))
plt.scatter(X_pca_cont[:, 0], X_pca_cont[:, 1], c=y, cmap='tab10', s=5)
plt.colorbar(label='Classe Real')
plt.title('PCA 2D — Apenas Variáveis Contínuas')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid(True, alpha=0.5)
plt.tight_layout()
plt.show()

print("""
As 44 variáveis binárias (Wilderness_Area e Soil_Type) foram excluídas nesta projeção
porque possuem baixa variância (valores 0 ou 1), o que pode distorcer a análise PCA.

→ O PCA com apenas variáveis contínuas proporciona uma visualização mais limpa e útil.
""")

print("=================================================================================================")

print("""
A escolha de incluir tanto as variáveis contínuas quanto as binárias no PCA foi feita para observar como a combinação de ambas as variáveis pode impactar a separação das classes. Enquanto as variáveis contínuas oferecem uma maior variabilidade, as variáveis binárias podem fornecer informações sobre a presença ou ausência de certos atributos que, combinadas com as variáveis contínuas, podem revelar padrões importantes.

Porém, é importante notar que as variáveis binárias têm baixa variância (apenas dois valores: 0 ou 1), e o PCA pode não explorar de maneira eficiente essas variáveis, uma vez que elas não contribuem muito para a variação dos dados. Mesmo assim, ao realizar o PCA com todas as variáveis, estamos observando como a redução de dimensionalidade afeta a separação das classes considerando a totalidade dos dados.

Esta abordagem ajuda a avaliar o comportamento do modelo tanto com a combinação das variáveis contínuas e binárias quanto com a utilização apenas das variáveis contínuas, permitindo uma análise comparativa de como diferentes variáveis impactam a separação entre as classes no espaço projetado.
""")

print("\n=== Fronteira de Decisão 3‑D (KNN) ===")
print("""
Objetivo: visualizar em 3 dimensões como o KNN separa as classes para três
variáveis contínuas. Escolhemos Elevation, Slope e Aspect porque:
• Elevation + Slope já mostrou boa separação 2‑D;
• Aspect acrescenta variação angular que dificulta a fronteira — bom exemplo.
""")

f1, f2, f3 = 'Elevation', 'Slope', 'Aspect'
X_3d      = X[[f1, f2, f3]]
X_3d_norm = StandardScaler().fit_transform(X_3d)
y_3d      = y

# Amostra p/ performance
sample = np.random.choice(len(X_3d_norm), 15000, replace=False)
X_s, y_s = X_3d_norm[sample], y_3d.iloc[sample]

knn3d = KNeighborsClassifier(n_neighbors=5, weights='distance').fit(X_s, y_s)

# grade em 3‑D (malha cúbica reduzida para economizar)
grid_lin = np.linspace(-3, 3, 30)
gx, gy, gz = np.meshgrid(grid_lin, grid_lin, grid_lin)
grid_points = np.c_[gx.ravel(), gy.ravel(), gz.ravel()]
grid_pred   = knn3d.predict(grid_points).reshape(gx.shape)

fig = go.Figure()

# adicionar nuvem real
fig.add_trace(go.Scatter3d(
    x=X_s[:,0], y=X_s[:,1], z=X_s[:,2],
    mode='markers',
    marker=dict(size=2, color=y_s, colorscale='Viridis', opacity=0.5),
    name='Amostras'
))
# adicionar superfícies finas da fronteira (isosurface)
fig.add_trace(go.Isosurface(
    x=gx.flatten(), y=gy.flatten(), z=gz.flatten(),
    value=grid_pred.flatten().astype(int),
    opacity=0.15,
    colorscale='Viridis',
    showscale=False,
    caps=dict(x_show=False, y_show=False, z_show=False),
    name='Fronteira'
))

fig.update_layout(
    title="Fronteira de Decisão 3‑D — KNN (Elevation, Slope, Aspect)",
    scene=dict(
        xaxis_title=f1, yaxis_title=f2, zaxis_title=f3
    )
)
fig.show()


In [ ]:
from sklearn.model_selection import StratifiedKFold

print("""
--------------------------
Teste de múltiplos valores de K (comentado devido ao tempo de execução)
--------------------------
Para datasets grandes, testar uma gama de valores de k pode ser extremamente demorado.
Com base na nossa situação atual, para otimizar o tempo de treinamento, optamos por usar
apenas o valor k=10 para continuar com o projeto, como explicamos abaixo.
""")

print("Observe o codigo comentado acima...")

# # Função para testar e plotar resultados de acordo com o tipo de peso
# def testar_knn(weight_type):
#     print(f"\n--- Testando KNN com weights='{weight_type}' ---\n")
#
#     k_values = range(1, 51)  # Testando k de 1 a 50
#     cv_scores = []
#
# # Por padrao, validacao cruzada usa Kfold -> Classes Balanceadas no dataset completo
#     for k in k_values:
#         knn = KNeighborsClassifier(n_neighbors=k, weights=weight_type)
#         scores = cross_val_score(knn, X_train_scaled, y_train, cv=10, scoring='accuracy', n_jobs=-1)
#         cv_scores.append(scores.mean())
#
#     # Plotar resultados
#     plt.figure(figsize=(10, 6))
#     plt.plot(k_values, cv_scores, marker='o', linestyle='dashed', label=f'weights="{weight_type}"')
#     plt.title(f'Validação Cruzada KNN — Pesos: {weight_type}', fontsize=16)
#     plt.xlabel('Número de Vizinhos (k)', fontsize=12)
#     plt.ylabel('Acurácia Média', fontsize=12)
#     plt.xticks(k_values)
#     plt.grid(True, alpha=0.5)
#     plt.legend()
#     plt.show()
#
#     best_k = k_values[cv_scores.index(max(cv_scores))]
#     print(f"Melhor valor de k para weights='{weight_type}': {best_k}")
#     print(f"Acurácia média: {max(cv_scores):.4f}\n")
#
# # Testar ambos os tipos de peso
# testar_knn('uniform')
# testar_knn('distance')

print("""
--------------------------
Versão para Demonstrar o teste de melhor K-vizinhos para o KNN de forma rápida
--------------------------
""")

print("\nDividindo uma amostra pequena do dataset em 80% treino / 20% teste (APENAS PARA DEMONSTRAR)…")
print("\nUsando 2000 amostras para ter numeros de classes suficientes para cv = 3 (splits) e acelerar a demonstracao da nossa ideia original...")

# Selecionar uma amostra pequena (2000 exemplos, por exemplo) para a demonstração
sample_idx = np.random.choice(len(X), size=2000, replace=False)
X_demo = X.iloc[sample_idx]
y_demo = y.iloc[sample_idx]

# Dividir a amostra em treino e teste
X_train_demo, X_test_demo, y_train_demo, y_test_demo = train_test_split(X_demo, y_demo, test_size=0.2, stratify=y_demo, random_state=42)

# Normalização para a amostra
scaler = StandardScaler()
X_train_demo_scaled = scaler.fit_transform(X_train_demo)
X_test_demo_scaled = scaler.transform(X_test_demo)

def testar_knn(weight_type):
    print(f"\n--- Testando KNN com weights='{weight_type}' e k variando de 1 a 50 ---\n")

    # Usando k de 1 a 50 para garantir que o teste seja rápido
    k_values = range(1, 51)  # Testando k de 1 a 50
    cv_scores = []

    # Usar 3 splits para a validação cruzada (Reduzido devido ao erro de poucas instâncias)
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

    # Explicação sobre o uso do StratifiedKFold
    print("StratifiedKFold faz a divisão dos dados em 3 partes (splits) (nesse contexto), garantindo que cada divisão preserve a distribuição das classes.")
    print("Estamos reduzindo os splits de 5 para 3, pois uma das classes tem apenas 4 instâncias (ao reduzir para 1000 amostras somente), o que causa o erro:")
    print("The least populated class in y has only 4 members, which is less than n_splits=5.")
    print("""
    Esse erro ocorre porque o KFold tenta dividir os dados em 5 partes (splits), mas uma das classes tem apenas 4 exemplos.
    Com isso, não é possível garantir que cada uma das 5 divisões tenha pelo menos uma instância dessa classe.
    Ao reduzir para 3 splits, garantimos que as 4 instâncias da classe rara possam ser distribuídas nas 3 partes, evitando o erro:
        Split 1: 1 instância da classe y
        Split 2: 1 instância da classe y
        Split 3: 2 instâncias da classe y
    """)

    # Explicação mais detalhada sobre o StratifiedKFold
    print("""
    O StratifiedKFold é uma versão modificada do KFold, projetada para garantir que a distribuição das classes seja mantida em cada uma das divisões (splits) durante a validação cruzada.
    Ao usar o StratifiedKFold, garantimos que a **proporção de instâncias de cada classe** seja aproximadamente a mesma em **todas as divisões** (splits), o que é fundamental quando temos **classes desbalanceadas** (com número desigual de instâncias).
    Isso é muito importante para evitar que o modelo seja treinado com um conjunto de dados que tenha **poucos exemplos de uma classe específica**, o que pode prejudicar a performance do modelo na previsão dessa classe.

    ### Como funciona o StratifiedKFold:
    1. **Estratificação**: O StratifiedKFold divide o conjunto de dados de forma que cada divisão (fold) contenha uma proporção representativa de cada classe. Por exemplo, se você tem 1000 exemplos, com 80% da classe A e 20% da classe B, cada fold terá aproximadamente 80% de instâncias da classe A e 20% de instâncias da classe B.
    2. **Proporção das classes**: Mesmo com **classes desbalanceadas**, o StratifiedKFold garante que cada fold tenha **uma boa representação de cada classe**, o que ajuda a treinar o modelo de maneira mais equilibrada.
    3. **Solução para o desbalanceamento**: Isso é especialmente importante quando você tem **classes com poucas instâncias**, como no caso de uma classe que tem apenas 4 exemplos. Usando o StratifiedKFold, você garante que essas 4 instâncias sejam bem distribuídas em todos os folds, sem que uma divisão fique sem exemplos dessa classe.

    ### Exemplos práticos de onde usar:
    1. **Dataset desbalanceado**: Se uma classe tiver muitas instâncias e outra tiver poucas, o StratifiedKFold ajuda a garantir que ambas as classes sejam bem representadas em todas as divisões.
    2. **Evitar o problema de ausência de classe**: Sem o StratifiedKFold, se uma classe tem apenas 4 instâncias, uma divisão de 5 partes pode acabar não tendo **nenhuma instância dessa classe** em algum dos folds. Isso prejudicaria a validação do modelo. Com o StratifiedKFold, isso é evitado, garantindo que todos os folds tenham amostras de todas as classes.

    ### No nosso contexto:
    Agora, com o **dataset reduzido para 1000 amostras**, as classes ficaram **desbalanceadas**, o que causou o problema que mencionamos anteriormente. Ou seja, a divisão de classes no conjunto de dados de 1000 amostras fez com que algumas classes tivessem apenas **poucas instâncias**, o que **exigiu o uso de StratifiedKFold** para garantir que a distribuição das classes fosse mantida.

    No entanto, é importante notar que não era suficiente apenas reduzir o número de splits de cv=5 para cv=3 para resolver o problema das classes com poucas instâncias. A maneira correta de resolver o problema de classes desbalanceadas é usar o StratifiedKFold. O StratifiedKFold garante que a distribuição proporcional das classes seja mantida em todos os folds, o que não é garantido pelo KFold, mesmo com a redução de splits.

    ### Código original: Todo o dataset (antes da redução para 1000 amostras):
    - **As classes eram balanceadas?**
      - No **dataset original completo**, se as classes forem **balanceadas**, o uso de **`cv=5`** funcionaria bem sem o `StratifiedKFold`. O modelo seria treinado e validado de forma justa, sem se preocupar com a representação de classes, porque as classes têm **um número semelhante de instâncias**.
      - Se as classes no dataset original são **bem balanceadas**, **não há necessidade de usar `StratifiedKFold`**.

    ### Conclusão:
    - O StratifiedKFold foi necessário no seu caso para garantir a distribuição proporcional das classes no dataset reduzido para 1000 amostras, uma vez que o desbalanceamento nas classes poderia afetar a validação cruzada.

    - A redução de cv=5 para cv=3 foi uma medida necessária para lidar com o número reduzido de instâncias de algumas classes, já que com 5 splits, seria difícil garantir a representatividade de todas as classes em cada fold. """)

    # Printando a quantidade de instâncias de cada classe no dataset demo
    print("\nQuantidade de instâncias de cada classe no dataset demo de 2000 amostras:")
    print(y_demo.value_counts())

    print("Para cada valor de K testado no KNN, o modelo será treinado e avaliado usando validação cruzada com 3 folds definidos. O resultado da avaliação será calculado em termo de acurácia média entre os 3 splits.")
    for k in k_values:
        knn = KNeighborsClassifier(n_neighbors=k, weights=weight_type)
        scores = cross_val_score(knn, X_train_demo_scaled, y_train_demo, cv=skf, scoring='accuracy', n_jobs=-1)
        cv_scores.append(scores.mean())

    print("""
    O que é **Cross-Validation (Validação Cruzada)?**

    A **Validação Cruzada** é uma técnica usada para avaliar o desempenho de um modelo de aprendizado de máquina. Ela ajuda a garantir que o modelo tenha um bom desempenho não apenas no conjunto de dados de treino, mas também em dados que ele **ainda não viu**, ou seja, em dados **de teste**. O objetivo é evitar que o modelo seja **overfitted** (ajustado excessivamente aos dados de treino) e garantir que ele generalize bem para novos dados.

    Por padrão, o método utilizado para validação cruzada é o **KFold**.

    ### O que o **KFold** faz?
    - O **KFold** divide o conjunto de dados em **K partes (splits)** aproximadamente iguais. O modelo é treinado **K-1 vezes** e testado uma vez em cada uma das divisões (folds). Cada divisão do conjunto de dados é utilizada como conjunto de teste uma vez, enquanto as outras são usadas para treinar o modelo.
    - O desempenho do modelo é então calculado como a **média das avaliações** realizadas em cada fold. Isso ajuda a fornecer uma estimativa mais robusta do desempenho do modelo em dados não vistos.
    - O **KFold** é uma ótima escolha quando as classes são **balanceadas** (ou seja, as classes têm uma quantidade semelhante de instâncias).

    ### O que é o **StratifiedKFold** e por que usá-lo?
    - O **StratifiedKFold** é uma **variante do KFold** projetada para garantir que as classes estejam **proporcionalmente representadas em cada fold**. Isso é essencial em datasets com **classes desbalanceadas** (onde uma classe tem significativamente mais exemplos que outra).
    - **Sem o StratifiedKFold**, o KFold pode criar folds em que algumas classes **não estão representadas**. Isso pode afetar negativamente a validação do modelo, pois o modelo pode não ser treinado nem testado adequadamente para essas classes minoritárias, resultando em uma avaliação imprecisa.
    - **Com o StratifiedKFold**, cada fold contém aproximadamente a mesma **proporção de instâncias de cada classe**, garantindo que o modelo seja treinado e validado de forma equilibrada, o que é especialmente importante quando temos **poucas instâncias de uma classe**.

    ### Como a escolha de `cv` (número de splits) afeta a validação?
    1. **Tamanho do Dataset**:
       - **Datasets pequenos**: Quando o dataset é pequeno, é comum usar um número menor de splits, como `cv=3` ou `cv=5`, para garantir que cada fold tenha **dados suficientes** para treinamento e teste.
       - **Datasets grandes**: Para datasets maiores, um número maior de folds, como `cv=10`, é preferido, pois isso ajuda a **aumentar a precisão** da estimativa da performance do modelo sem sacrificar a quantidade de dados de treinamento.
    2. **Número de Splits (cv)**:
       - **Muito grande (`cv=10`)**: O número de instâncias por fold pode ser pequeno, o que pode **aumentar a variabilidade** dos resultados e tornar o desempenho mais **instável**.
       - **Muito pequeno (`cv=2`)**: Isso pode resultar em folds com **dados insuficientes** para uma avaliação justa, prejudicando a performance.
    3. **Impacto do Tempo de Execução**:
       - **Maior `cv` leva a maior tempo de execução**, pois o modelo será treinado e testado várias vezes, aumentando o número de **iteracões**.

    ### Resumo e Conclusão:
    - **Cross-validation** é uma técnica essencial para garantir que um modelo de aprendizado de máquina generalize bem para novos dados.
    - Usamos **KFold** por padrão, mas o **StratifiedKFold** deve ser preferido quando temos **classes desbalanceadas**, pois ele garante que todas as classes sejam representadas proporcionalmente em todos os folds.
    - A escolha do número de splits (`cv`) deve equilibrar a necessidade de **precisão na avaliação** e o **tempo de processamento**. Para datasets pequenos, `cv=3` ou `cv=5` pode ser ideal. Para datasets maiores, valores como `cv=10` podem ser usados para uma avaliação mais robusta e precisa.

    **Importante**: **Se um fold não contiver instâncias de uma classe**, o modelo **não será testado** para essa classe durante a validação cruzada, o que **não faz sentido** e pode prejudicar o desempenho. O modelo **não aprenderá** a classificar essa classe corretamente, o que resulta em uma avaliação falha e uma baixa capacidade de generalização para dados de classes ausentes.
    """)

    print("""
    Para escolher o valor de `cv` (número de splits) adequado, algumas considerações devem ser feitas, principalmente em relação ao tamanho do dataset e o equilíbrio entre as classes.

    1. **Tamanho do Dataset**:
       - Se o dataset for pequeno, é comum usar um número menor de splits, como `cv=3` ou `cv=5`. Isso ajuda a garantir que cada fold tenha dados suficientes para treinar e testar o modelo.
       - Para datasets grandes, como o caso de usar o dataset completo, um número maior de splits (como `cv=10`) pode ser apropriado, pois isso permite uma melhor avaliação do modelo, aumentando a precisão da estimativa da performance.

    2. **Impacto do Número de Splits**:
       - Se a quantidade de splits (`cv`) for muito grande, como `cv=10`, cada fold pode ter um número muito pequeno de dados para treinamento, o que pode levar a uma estimativa mais instável da performance do modelo.
       - Por outro lado, se `cv` for muito pequeno (como `cv=2`), o modelo pode não ser validado adequadamente em diferentes partes do conjunto de dados.

    3. **Tempo de Execução**:
       - A escolha de `cv` também é influenciada pelo tempo de processamento. Um número maior de folds aumenta o tempo de execução, porque o modelo será treinado e avaliado mais vezes.

    ### Conclusão:
    - No nosso contexto, dado que o dataset foi reduzido para 2000 amostras e observamos que o número de splits de 3 funcionou bem, usamos `cv=3` com o `StratifiedKFold` para garantir que todas as classes estivessem bem representadas.
    - Para o **dataset completo**, optariamos em usar `cv=10`, já que isso parece proporcionar um bom equilíbrio entre avaliação robusta e tempo de execução, além de que parece uma quantidade adequada para o tamanho do dataset.
    """)

    # Plotar resultados de forma simplificada para demonstração
    plt.figure(figsize=(10, 6))
    plt.plot(k_values, cv_scores, marker='o', linestyle='dashed', label=f'weights="{weight_type}"')
    plt.title(f'Validação Cruzada KNN — Pesos: {weight_type} (k=1 a 50) / 2000 amostras', fontsize=16)
    plt.xlabel('Número de Vizinhos (k)', fontsize=12)
    plt.ylabel('Acurácia Média', fontsize=12)
    plt.xticks(k_values)
    plt.grid(True, alpha=0.5)
    plt.legend()
    plt.show()

    # Mostrar o melhor valor de k
    best_k = k_values[cv_scores.index(max(cv_scores))]
    print(f"Melhor valor de k para weights='{weight_type}': {best_k}")
    print(f"Acurácia média para k={best_k}, weights='{weight_type}': {max(cv_scores):.4f}\n")

# Testar ambos os tipos de peso com k variando de 1 a 50
testar_knn('uniform')
testar_knn('distance')

print("Por que, mesmo usando `random_state`, os resultados do melhor valor de K (e a acurácia) podem mudar?")
print("""
Embora o `random_state` seja fixo e garanta a consistência na divisão dos dados ( dados aleatorios para mesmo resultado sempre ) durante a validação cruzada, o tipo de **KFold** utilizado (simples ou **StratifiedKFold**) pode impactar significativamente os resultados.

- **`random_state`**: É uma semente de aleatoriedade que controla o processo de divisão dos dados ou a inicialização de parâmetros. Quando o **`random_state`** é fixo, ele garante que a **divisão dos dados** ou a **inicialização dos centros no K-Means** sejam **sempre as mesmas** a cada execução. Isso proporciona **resultados reproduzíveis** e evita variações de desempenho apenas por conta de aleatoriedade.

- **KFold**: Este método divide os dados de maneira simples, em partes iguais, mas **não garante a distribuição proporcional das classes**. Em datasets com **classes desbalanceadas**, isso pode fazer com que algumas classes sejam **sub-representadas em alguns folds**, o que pode prejudicar a avaliação do modelo.

- **StratifiedKFold**: Esse método, por outro lado, **garante que a distribuição das classes seja proporcional em cada divisão (fold)**. Isso é essencial para garantir que **todas as classes sejam bem representadas em todos os folds**, especialmente em casos de datasets com **classes desbalanceadas**.

Portanto, mesmo com o **`random_state`** fixo, a escolha entre **KFold** e **StratifiedKFold** pode alterar o desempenho do modelo. A média de acurácia e, consequentemente, o número ideal de **K** para o modelo podem variar, pois o **StratifiedKFold** ajuda a garantir uma validação mais equilibrada, enquanto o **KFold simples** pode não ser tão robusto, especialmente em datasets com desequilíbrio de classes.

Além disso, o **`random_state`** também é utilizado em outros algoritmos, como o **K-Means**, onde ele controla a **inicialização dos centros de clusters**. Mesmo com o `random_state` fixo, o número de clusters e a qualidade das predições podem mudar dependendo do método de validação cruzada (KFold vs. StratifiedKFold), pois as distribuições de classes e os dados usados para treino/teste podem ser diferentes em cada divisão.

O uso de **`random_state`** é crucial para garantir **reprodutibilidade** dos resultados, mas o tipo de **KFold** escolhido pode influenciar a avaliação do modelo dependendo da natureza dos dados.
""")

# --------------------------
# Uso do k=10 para continuar com o projeto
# --------------------------

print("""
Optamos por usar k=10 para continuar com o projeto devido ao tempo de processamento com datasets grandes e pela dificuldade em determinar o melhor valor de k neste momentoe nao sabermos como arrumar isso.
Embora o valor ideal de k possa ser encontrado por validação cruzada com diversos valores de k, isso exigiria mais tempo e poder computacional, o que não é viável neste momento. O filipe ficou rodando por 30 minutos e não obteve resultados satisfatórios...

Percebemos que, ao testarmos com 2000 amostras, o valor de k se manteve em torno de 5, oferecendo uma boa média de acurácia. Com base nisso, para o **dataset completo**, decidimos utilizar **k=10**. Acreditamos que, conforme o número de amostras aumenta, o valor de k também deve ser maior, o que tende a proporcionar uma melhor generalização e estabilidade ao modelo.

Sabemos que o uso de k=5 ou k=10 não é o ideal, pois valores menores ou maiores podem oferecer um desempenho melhor dependendo da distribuição dos dados. Contudo, por questões práticas, seguimos com k=10 para dar continuidade ao desenvolvimento do projeto.
""")

In [ ]:
print("\nDividindo em 80% treino / 20% teste (estratificado)…")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Normalização
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ---------------- UNIFORME ----------------
print("Modelo com Pesos Uniformes (80% Treino e 20% Teste)\n")
knn_uniform = KNeighborsClassifier(n_neighbors=10, weights='uniform')
knn_uniform.fit(X_train_scaled, y_train)
y_pred_uniform = knn_uniform.predict(X_test_scaled)
f1_uniform_1 = classification_report(y_test, y_pred_uniform, output_dict=True)['weighted avg']['f1-score']

print("Relatório de Classificação (Uniforme):\n", classification_report(y_test, y_pred_uniform))
print("Matriz de Confusão (Uniforme):\n", confusion_matrix(y_test, y_pred_uniform))
print("Acurácia (Uniforme):", accuracy_score(y_test, y_pred_uniform))
print(f"F1-Score (Uniforme): {f1_uniform_1:.4f}")

# ---------------- DISTÂNCIA ----------------
print("\nModelo com Pesos por Distância (80% Treino e 20% Teste)\n")
knn_distance = KNeighborsClassifier(n_neighbors=10, weights='distance')
knn_distance.fit(X_train_scaled, y_train)
y_pred_distance = knn_distance.predict(X_test_scaled)
f1_distance_1 = classification_report(y_test, y_pred_distance, output_dict=True)['weighted avg']['f1-score']

print("Relatório de Classificação (Distância):\n", classification_report(y_test, y_pred_distance))
print("Matriz de Confusão (Distância):\n", confusion_matrix(y_test, y_pred_distance))
print("Acurácia (Distância):", accuracy_score(y_test, y_pred_distance))
print(f"F1-Score (Distância): {f1_distance_1:.4f}")

print("Analise das metricas e motivos de separar ; testar unform e distance abaixo")


In [ ]:
# Normalizar a base inteira
X_scaled = scaler_full.fit_transform(X)

# ----------- UNIFORM -------------
print("\nModelo com a Base Inteira — Pesos Uniformes\n")

knn_uniform = KNeighborsClassifier(n_neighbors=10, weights='uniform')
knn_uniform.fit(X_scaled, y)
y_pred_uniform = knn_uniform.predict(X_scaled)
f1_uniform_2 = classification_report(y, y_pred_uniform, output_dict=True)['weighted avg']['f1-score']

print("Relatório de Classificação (Uniform):\n", classification_report(y, y_pred_uniform))
print("Matriz de Confusão (Uniform):\n", confusion_matrix(y, y_pred_uniform))
print("Acurácia (Uniform):", accuracy_score(y, y_pred_uniform))
print(f"F1-Score (Uniforme): {f1_uniform_2:.4f}")

# ----------- DISTANCE -------------
print("\nModelo com a Base Inteira — Pesos por Distância\n")

knn_distance = KNeighborsClassifier(n_neighbors=10, weights='distance')
knn_distance.fit(X_scaled, y)
y_pred_distance = knn_distance.predict(X_scaled)
f1_distance_2 = classification_report(y, y_pred_distance, output_dict=True)['weighted avg']['f1-score']  # Usando a base completa para avaliação

print("Relatório de Classificação (Distance):\n", classification_report(y, y_pred_distance))
print("Matriz de Confusão (Distance):\n", confusion_matrix(y, y_pred_distance))
print("Acurácia (Distance):", accuracy_score(y, y_pred_distance))
print(f"F1-Score (Distância): {f1_distance_2:.4f}")




print("""
Comparação com a base completa (sem divisão treino/teste):
- Ambos os modelos tendem a ter acurácia superestimada.
- A versão com pesos por distância pode se sair melhor em regiões onde há sobreposição de classes,
  pois ela dá mais importância aos vizinhos mais próximos.

### Explicação:
Quando utilizamos a base inteira (sem dividir entre treino e teste), o modelo tem acesso a todas as instâncias durante o treinamento e a validação, o que pode levar a um resultado otimista (superestimado), já que o modelo "vê" o conjunto de dados inteiro, incluindo os exemplos que são usados para validar o modelo. Isso pode não refletir o desempenho real do modelo em dados não vistos.

A escolha de **pesos uniformes** ou **pesos por distância** no KNN pode afetar a performance do modelo, principalmente quando há **sobreposição entre classes**. Com **pesos por distância**, vizinhos mais próximos têm mais influência na decisão final, o que pode ser vantajoso em áreas de sobreposição, pois o modelo tende a "dar mais peso" aos exemplos mais relevantes.

No entanto, ao usar o modelo no conjunto de dados inteiro, **a acurácia pode ser inflada**, já que o modelo já tem acesso a todas as instâncias durante o treinamento e validação.

---

Comparação entre os pesos no KNN:

- **'uniform'**: todos os vizinhos têm o mesmo peso na decisão.
- **'distance'**: vizinhos mais próximos influenciam mais.

### Explicação dos Pesos:
1. **Pesos Uniformes ('uniform')**:
   - No KNN com pesos uniformes, **todos os vizinhos** de um ponto de teste têm a **mesma influência na decisão** de classificação, independentemente da distância. Ou seja, o modelo simplesmente conta quantos vizinhos de cada classe existem no conjunto de vizinhos mais próximos e escolhe a classe que aparece com maior frequência.
   - **Vantagens**: Pode ser mais estável e simples, especialmente em áreas onde as classes estão bem separadas.
   - **Desvantagens**: Em áreas com **muita sobreposição de classes**, o modelo pode cometer mais erros, já que todos os vizinhos têm o mesmo peso, mesmo que alguns sejam muito mais distantes.

2. **Pesos por Distância ('distance')**:
   - No KNN com pesos por distância, **os vizinhos mais próximos** têm **maior influência** na decisão de classificação. Ou seja, a contribuição de cada vizinho é ponderada pela sua distância ao ponto de teste, com vizinhos mais próximos tendo um peso maior.
   - **Vantagens**: Pode ser mais eficaz quando as classes estão **muito próximas** umas das outras, pois o modelo dá mais importância aos vizinhos mais próximos, o que pode ajudar a reduzir o impacto do ruído nas bordas das classes.
   - **Desvantagens**: Pode ser mais sensível a **outliers** e ao **ruído** nos dados, pois um vizinho distante com um valor muito diferente pode ter grande influência na decisão.

### Impacto do Tipo de Peso:
A escolha entre **pesos uniformes** e **pesos por distância** pode impactar significativamente a **acurácia** do modelo, dependendo da **distribuição dos dados**. Para classes que estão muito próximas ou com muita sobreposição, **pesos por distância** podem melhorar o desempenho, já que a decisão será mais influenciada pelos vizinhos mais próximos e mais relevantes. Já em datasets com **classes bem separadas**, o uso de **pesos uniformes** pode ser mais eficaz e gerar resultados igualmente bons, mas de forma mais estável.

---

Explicação das Métricas de Avaliação:

1. **Acurácia** (`accuracy_score`):
   - A **acurácia** é uma das métricas mais simples e comuns, e representa a **proporção de previsões corretas** (tanto positivos quanto negativos) em relação ao total de previsões feitas.
   - **Fórmula**:
     \[
     \text{Acurácia} = \frac{\text{Número de Previsões Corretas}}{\text{Total de Previsões}}
     \]
   - Embora fácil de entender, a **acurácia** pode ser enganosa em datasets desbalanceados, pois o modelo pode simplesmente prever a classe majoritária e obter uma alta acurácia, mesmo sem aprender corretamente a classificar a classe minoritária.

2. **Relatório de Classificação** (`classification_report`):
   - O **relatório de classificação** fornece **precisão**, **recall** e **F1-score** para cada classe individualmente e uma **média ponderada** dessas métricas.
     - **Precisão**: A proporção de **verdadeiros positivos** sobre o total de previsões positivas (TP / (TP + FP)).
     - **Recall**: A proporção de **verdadeiros positivos** sobre o total de exemplos que pertencem à classe (TP / (TP + FN)).
     - **F1-score**: A média harmônica entre precisão e recall, que busca um equilíbrio entre as duas. É especialmente útil quando há um **desequilíbrio de classes**.
   - **Fórmula do F1-score**:
     \[
     F1 = 2 \times \frac{\text{Precisão} \times \text{Recall}}{\text{Precisão} + \text{Recall}}
     \]
   - Essa métrica é muito útil para medir o desempenho do modelo quando as classes estão desbalanceadas, já que ela leva em consideração tanto a precisão quanto a capacidade do modelo de **capturar as instâncias da classe minoritária**.

3. **Matriz de Confusão** (`confusion_matrix`):
   - A **matriz de confusão** mostra a quantidade de **verdadeiros positivos (TP)**, **falsos positivos (FP)**, **falsos negativos (FN)** e **verdadeiros negativos (TN)**.
   - Essa métrica ajuda a entender **onde o modelo está errando**. Por exemplo:
     - Se o modelo está fazendo muitos **falsos negativos**, isso pode indicar que ele não está identificando bem a classe positiva.
     - Se há muitos **falsos positivos**, o modelo pode estar confundindo instâncias de uma classe com outra.
   - A matriz de confusão fornece insights importantes para melhorar o modelo, já que revela como ele está se comportando para cada classe.

4. **F1-Score**:
   - O **F1-score** é uma métrica combinada que tenta equilibrar a **precisão** e o **recall**. Ele é especialmente útil quando temos **classes desbalanceadas**, pois ajuda a avaliar o desempenho do modelo na previsão da classe minoritária sem ser tão influenciado pela classe majoritária.
   - A **média ponderada** do F1-score leva em consideração o número de instâncias de cada classe para dar mais peso às classes com mais instâncias.
""")
